In [3]:
import numpy as np

In [4]:
def hex2bin(c):
    return np.binary_repr(int(c, base=16), width=4)

def decode2bin(s):
    num = ""
    for c in s:
        num += hex2bin(c)
    return num

def pack(arr, wordsize=8):
    npack = wordsize - len(arr)
    return np.hstack([np.zeros(npack, dtype=int), arr])

def pack8(arr):
    return np.packbits(pack(arr, wordsize=8))[0]

def pack16(arr):
    bytearr = np.packbits(pack(arr, wordsize=16))
    return 256 * bytearr[0] + bytearr[1]

def arr2str(arr):
    return ''.join([f"{i}" for i in arr])

In [6]:
test_str = "D2FE28"
decode2bin(test_str)

'110100101111111000101000'

In [7]:
decode2bin("38006F45291200")

'00111000000000000110111101000101001010010001001000000000'

In [138]:
def mainparser(binstr):
    output = 0
    if len(binstr) > 6:
        ver = int(binstr[:3], base=2)
        typ = int(binstr[3:6], base=2)
        binstr = binstr[6:]
        if typ == 4:
            binstr, output = literal(binstr)
            print(f"literal returned {output}")
        else:
            if binstr[0] == "0":
                literals = []
                subpacketlen = int(binstr[1:16], 2)
                binstr = binstr[16:]
                subpackets = binstr[:subpacketlen]
                binstr = binstr[subpacketlen:]
                while True:
                    subpackets, v = mainparser(subpackets)
                    literals += [v]
                    if len(subpackets) < 6:
                        break
                output = operation(literals, typ)
            else:
                literals = []
                subcount = int(binstr[1:12], base=2)
                binstr = binstr[12:]
                for _ in range(subcount):
                    binstr, v = mainparser(binstr)
                    literals += [v]
                output = operation(literals, typ)

    return binstr, output

def literal(binstr):
    chunks = []
    chunking = True
    while chunking:
        chunks += [binstr[:5]]
        binstr = binstr[5:]
        if chunks[-1][0] == '0':
            chunking = False
    l = int(''.join([x[1:] for x in chunks]), 2)
    return binstr, l

def prod(literals):
    out = literals[0]
    for x in literals[1:]:
        out *= x
    return out

def operation(literals, op):
    if op == 0:
        print(f"add {literals}")
        return sum(literals)
    elif op == 1:
        print(f"prod {literals}")
        return prod(literals)
    elif op == 2:
        print(f"min {literals}")
        return min(literals)
    elif op == 3:
        print(f"max {literals}")
        return max(literals)
    elif op == 5:
        print(f"gt {literals}")
        return 1 if literals[0] > literals[1] else 0
    elif op == 6:
        print(f"lt {literals}")
        return 1 if literals[0] < literals[1] else 0
    elif op == 7:
        print(f"eq {literals}")
        return 1 if literals[0] == literals[1] else 0

In [141]:
mainparser(decode2bin("9C0141080250320F1802104A08"))

literal returned 1
literal returned 3
add [1, 3]
literal returned 2
literal returned 2
prod [2, 2]
eq [4, 4]


('00', 1)

In [125]:
mainparser(decode2bin(test_str))

ver = 6, typ = 4
literal returned 2021


(6, '000')

In [126]:
mainparser(decode2bin("38006F45291200"))

ver = 1, typ = 6
ver = 6, typ = 4
literal returned 10
ver = 2, typ = 4
literal returned 20


(9, '0000000')

In [127]:
mainparser(decode2bin("EE00D40C823060"))

ver = 7, typ = 3
ver = 2, typ = 4
literal returned 1
ver = 4, typ = 4
literal returned 2
ver = 1, typ = 4
literal returned 3


(14, '00000')

In [128]:
mainparser(decode2bin("8A004A801A8002F478"))

ver = 4, typ = 2
ver = 1, typ = 2
ver = 5, typ = 2
ver = 6, typ = 4
literal returned 15


(16, '000')

In [129]:
mainparser(decode2bin("620080001611562C8802118E34"))

ver = 3, typ = 0
ver = 0, typ = 0
ver = 0, typ = 4
literal returned 10
ver = 5, typ = 4
literal returned 11
ver = 1, typ = 0
ver = 0, typ = 4
literal returned 12
ver = 3, typ = 4
literal returned 13


(12, '00')

In [130]:
mainparser(decode2bin("C0015000016115A2E0802F182340"))

ver = 6, typ = 0
ver = 0, typ = 0
ver = 0, typ = 4
literal returned 10
ver = 6, typ = 4
literal returned 11
ver = 4, typ = 0
ver = 7, typ = 4
literal returned 12
ver = 0, typ = 4
literal returned 13


(23, '000000')

In [131]:
mainparser(decode2bin("A0016C880162017C3686B18A3D4780"))

ver = 5, typ = 0
ver = 1, typ = 0
ver = 3, typ = 0
ver = 7, typ = 4
literal returned 6
ver = 6, typ = 4
literal returned 6
ver = 5, typ = 4
literal returned 12
ver = 2, typ = 4
literal returned 15
ver = 2, typ = 4
literal returned 15


(31, '0000000')

In [142]:
with open("input.txt", 'r') as fp:
    data = fp.readline().strip()
mainparser(decode2bin(data))

literal returned 206
literal returned 176
add [206, 176]
literal returned 14
literal returned 12148392
literal returned 68454822
max [12148392, 68454822]
literal returned 2967250
literal returned 95
lt [2967250, 95]
literal returned 1247339414
prod [0, 1247339414]
literal returned 4731280
literal returned 49792
literal returned 3454
eq [49792, 3454]
prod [4731280, 0]
literal returned 732272
literal returned 10
literal returned 13
literal returned 6
add [10, 13, 6]
literal returned 2
literal returned 8
literal returned 11
add [2, 8, 11]
gt [29, 21]
literal returned 106
prod [1, 106]
literal returned 11
literal returned 11969338
min [11, 11969338]
literal returned 286428
max [286428]
literal returned 168
literal returned 14637
literal returned 185
gt [14637, 185]
literal returned 3022
prod [1, 3022]
literal returned 2501
literal returned 60221721728
literal returned 1973125485
literal returned 250571
add [2501, 60221721728, 1973125485, 250571]
literal returned 114
literal returned 193
li

('0000000', 200476472872)